In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import welch
import scipy

In [6]:
channels = ['ch_'+str(i) for i in range(1,15)]

In [7]:
def cal_psd(data):
    eeg_channels = data.values.T  # Transpose the DataFrame to have channels in rows

    # Define parameters for PSD calculation
    window_size = 32  # Window size in seconds
    window_overlap = 0.0039  # Window overlap in percentage

    # Calculate PSD for each channel
    psds = []
    freqs = None

    for channel in eeg_channels:
        # Calculate PSD using Welch's method
        f, psd = welch(channel, fs=128, window='hanning',
                       nperseg=int(window_size * 128),
                       noverlap=int(window_overlap * window_size * 128))
        psds.append(psd)

    # Convert PSDs to a numpy array
    return np.array(psds)

In [8]:
import os

directory = "clean_data"   #one time clean other time normalize
for filename in os.listdir(directory):
    data = pd.read_csv(f'{directory}/{filename}')
    label = data['label'][0]
    data.drop(['label'] , axis = 1 , inplace = True)
    psds = cal_psd(data).T
    df = pd.DataFrame(psds , columns = channels)
    df['label'] = label
    df.to_csv(f'psd/{filename[:-4]}.csv',index = False)
       